# Autonomous Trading Brain - Kaggle Training Pipeline

This notebook sets up the environment, installs dependencies (including TA-Lib), prepares the data, and runs the training loop on GPU.

## 1. Environment Setup & TA-Lib Installation
Compiling TA-Lib C-library (required for technical indicators).

In [ ]:
import os
import sys

# Install system build tools
!apt-get update > /dev/null
!apt-get install -y build-essential > /dev/null

# Download and compile TA-Lib
if not os.path.exists('/usr/lib/libta_lib.so'):
    print("Downloading TA-Lib source...")
    !wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    !tar -xzf ta-lib-0.4.0-src.tar.gz
    
    print("Compiling TA-Lib (this takes ~2 mins)...")
    os.chdir('ta-lib')
    !./configure --prefix=/usr > /dev/null
    !make > /dev/null
    !make install > /dev/null
    os.chdir('..')
    print("TA-Lib installed successfully!")
else:
    print("TA-Lib already installed.")

## 2. Project Setup
Copying code from Input to Working directory to allow execution and modification.

In [ ]:
import shutil
from pathlib import Path

# CONFIG: Change this to match your uploaded dataset name
INPUT_DIR = Path('/kaggle/input/xtitan-project/xtitan') 
WORKING_DIR = Path('/kaggle/working/xtitan')

if not WORKING_DIR.exists():
    print(f"Copying project from {INPUT_DIR} to {WORKING_DIR}...")
    # Copy ignoring git/venv/cache if they exist
    shutil.copytree(INPUT_DIR, WORKING_DIR, 
                    ignore=shutil.ignore_patterns('venv', '__pycache__', '.git', '*.optimize', 'data_cache'))
    print("Project setup complete.")
else:
    print("Project already in working directory.")

os.chdir(WORKING_DIR)

## 3. Install Python Dependencies

In [ ]:
!pip install -r requirements.txt
!pip install TA-Lib

## 4. Data Preparation
**Check**: Do we need to download data or did you upload it?

In [ ]:
# Option A: Link uploaded data (if you uploaded data_cache as a dataset)
UPLOADED_DATA = Path('/kaggle/input/xtitan-data/data_cache')
LOCAL_DATA = Path('data_cache')

if UPLOADED_DATA.exists():
    print(f"Found uploaded data at {UPLOADED_DATA}. Linking...")
    if LOCAL_DATA.exists():
        shutil.rmtree(LOCAL_DATA)
    shutil.copytree(UPLOADED_DATA, LOCAL_DATA)
    print("Data linked successfully.")

else:
    print("No uploaded data found. Downloading from Deriv...")
    # Ensure .env exists or set vars (Deriv API doesn't need key for public historical data usually, 
    # but if your script needs it, you should set secrets in Kaggle)
    
    # Example download command (adjust symbol/months)
    !python scripts/download_data.py --months 12 --symbol R_100 --output data_cache

## 5. System Validation
Running pre-flight checks before training.

In [ ]:
!python pre_training_validation.py

## 6. Training
Running the main training loop.

In [ ]:
# Clear previous logs if needed
# !rm -rf logs/

# Run training
!python scripts/train.py

## 7. Artifact Archival
Zip results for download.

In [ ]:
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_name = f"training_artifacts_{timestamp}"

print("Archiving checkpoints and logs...")
shutil.make_archive(archive_name, 'zip', root_dir='.', base_dir='checkpoints')
shutil.make_archive(f"{archive_name}_logs", 'zip', root_dir='.', base_dir='logs')

print(f"Created {archive_name}.zip")